In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import math
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics

Using TensorFlow backend.


In [2]:
#tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 20
pd.options.display.float_format = '{:.1f}'.format

semesters = pd.read_csv('terminsstatus_17.txt', sep = '\t')
semesters.head()

,program,lopnr,female,kull,termin,tnr,status,poang_p,kullnr,ptnr
0,E,1,0,20002,20002,15,-7,6.0,15,1
1,W,2,1,20112,20112,37,-7,15.0,37,1
2,D,3,1,20032,20032,21,-7,16.5,21,1
3,N,4,1,20112,20112,37,-7,30.0,37,1
4,W,5,1,20112,20112,37,-7,0.0,37,1


In [3]:
sorted_on_start_year = semesters.sort_values(by = "kull")
soty = sorted_on_start_year
poang_p = soty['poang_p']

In [17]:
mean = 0
rmse = 0
acc = 0
mse = 0
for i,p in enumerate(poang_p):
    mse += np.power(p-mean,2)
    acc += p
    mean = acc/(i+1)


rmse = mse/poang_p.shape[0]
rmse = np.sqrt(rmse)

print(rmse) 

14.87464250441758


In [13]:
program_semesters = dict()

program = soty.program.unique()
for p in program:
    ##program_means[p] = semesters[semesters.program == p].poang_p.mean()
    program_semesters[p] = soty[soty.program == p].poang_p


rmse = 0
mean = 0
acc = 0
rmse_temp = 0
for ps in program_semesters:
    val = program_semesters.get(ps)
    mean = 0
    acc = 0
    rmse_temp = 0
    for i, p in enumerate(val):
        rmse_temp += np.power(p-mean, 2)
        acc += p
        mean = acc/(i+1)
    rmse_temp = rmse_temp/val.shape[0]
    rmse_temp = np.sqrt(rmse_temp)
    rmse += rmse_temp
    
rmse /= len(program_semesters)

print(rmse) 

14.363012154708152


In [19]:
boy_points = soty[soty.female==0].poang_p
girl_points = soty[soty.female==1].poang_p



girl_rmse = 0
girl_mean = 0
girl_acc = 0
for i, gp in enumerate(girl_points):
     girl_rmse += np.power(gp-girl_mean,2)
     girl_acc += gp
     girl_mean = girl_acc/(i+1)
girl_rmse /= girl_points.shape[0]
girl_rmse = np.sqrt(girl_rmse)
print(girl_rmse) # 13.31

14.753389114226874


In [18]:
boy_rmse = 0
boy_mean = 0
boy_acc = 0
for j, bp in enumerate(boy_points):
    boy_rmse += np.power(bp-boy_mean,2)
    boy_acc = 0
    boy_mean = boy_acc/(j+1)
boy_rmse /= boy_points.shape[0]
boy_rmse = np.sqrt(boy_rmse)

print(boy_rmse) 
rmse = (girl_rmse + boy_rmse)/2
print(rmse) 


21.493375597714618
18.123382355970747


In [20]:
delay_one_term = lambda x: x.shift(+1)
prev_semesters = semesters.apply(delay_one_term)
mean_points = semesters.poang_p.mean()

prev = []
for i,s in enumerate(semesters.lopnr):
    if s == prev_semesters.at[i,'lopnr']:
        prev.append(prev_semesters.at[i, 'poang_p'])
    else:
        prev.append(mean_points)
prev_points_pd = pd.DataFrame(prev)
semesters['prev_points'] = prev_points_pd


In [24]:
rmse = 0
mean = 0
acc = 0
count = 0

p1 = semesters.poang_p
p2 = semesters.prev_points

print(p1.head())
print(p2.head())

for i, p in enumerate(p1):
    rmse += np.power(p-p2[i], 2)
rmse /= len(p1)
rmse = np.sqrt(rmse)

rmse

0    6.0
1   15.0
2   16.5
3   30.0
4    0.0
Name: poang_p, dtype: float64
0   16.1
1   16.1
2   16.1
3   16.1
4   16.1
Name: prev_points, dtype: float64


13.820403228493069